In [8]:
import rasterio
from rasterio.mask import mask

In [9]:
import geopandas as gpd

In [11]:
shapefile = gpd.read_file(r"C:\Users\Nazia\research_ntl\trial\clip_shp\clip_houston_part.shp")

In [12]:
geoms = shapefile.geometry.values

In [13]:
geometry = geoms[0]

In [14]:
from shapely.geometry import mapping

In [15]:
geoms = [mapping(geoms[0])]

In [16]:
with rasterio.open(r"C:\Users\Nazia\research_ntl\trial\ntl_Mosaic_0201.tif") as src:
    out_image, out_transform = mask(src, geoms, crop=True)

In [17]:
no_data=src.nodata

In [18]:
data = out_image[0,:,:]

In [19]:
import numpy as np
row, col = np.where(data != no_data) 
elev = np.extract(data != no_data, data)

In [20]:
from rasterio import Affine 

In [21]:
T1 = out_transform * Affine.translation(0.5, 0.5)

In [22]:
rc2xy = lambda r, c: (c, r) * T1 

In [23]:
d = gpd.GeoDataFrame({'col':col,'row':row,'1000':elev})

In [24]:
d['x'] = d.apply(lambda row: rc2xy(row.row,row.col)[0], axis=1)
d['y'] = d.apply(lambda row: rc2xy(row.row,row.col)[1], axis=1)

<ipython-input-22-5ff012c45039>:1: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  rc2xy = lambda r, c: (c, r) * T1


In [25]:
from shapely.geometry import Point
d['geometry'] =d.apply(lambda row: Point(row['x'], row['y']), axis=1)

C:\Users\Nazia\AppData\Local\ESRI\conda\envs\arcgispro-py3-new\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [32]:
d.head(7)

,col,row,1000,x,y,geometry
0,0,0,0,-95.964583,30.16875,POINT (-95.96458 30.16875)
1,1,0,0,-95.960417,30.16875,POINT (-95.96042 30.16875)
2,2,0,0,-95.956250,30.16875,POINT (-95.95625 30.16875)
3,3,0,0,-95.952083,30.16875,POINT (-95.95208 30.16875)
4,4,0,0,-95.947917,30.16875,POINT (-95.94792 30.16875)
5,5,0,0,-95.943750,30.16875,POINT (-95.94375 30.16875)
6,6,0,0,-95.939583,30.16875,POINT (-95.93958 30.16875)


In [28]:
d.to_file(r"C:\Users\Nazia\research_ntl\trial\result", driver='ESRI Shapefile')